In [1]:
import wandb
import models.future_model
from lightning.pytorch.loggers import WandbLogger
import os
from lightning import Trainer
import datasets
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from models.necks import *

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
FatNeck = lambda h, v: LSTMNeck(h, v, 1, True, 128, 2)

In [4]:
wandb.login(key=os.environ['WANDB_API_KEY'], relogin=True)
wandb_logger = WandbLogger(
    name='base_run',
    project='LAISR_FutureGPT2',
    log_model='all',
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/wwu/.netrc


In [5]:
dataset = datasets.load_from_disk('/corpus/msmarco/msmarco_GPT2_64tokens_100k').with_format('torch')
loaders = {
    split: DataLoader(dataset[split], batch_size=128) 
    for split in ['train', 'val', 'test']
}

# use small val set; o/w too slow
loaders['val'] =  DataLoader(
    datasets.load_from_disk('/corpus/msmarco/msmarco_GPT2_64tokens_100k').with_format('torch')['val'],
    batch_size=128,
)

In [6]:
lr_callback = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(
    dirpath="/checkpoints/",
    # can't zero pad global_step b/c the logger casts it to a float >:(((
    filename="gpt2-test_run-{epoch:02d}-{global_step:.0f}-{train_total_loss:.2f}",
    #every_n_epochs=0.1,
    #every_n_train_steps=10_000,
    every_n_train_steps=10,
    #save_last=True,
    save_top_k=-1,
    monitor='global_step',
    mode='max',
    save_on_train_epoch_end=True,
)

In [7]:
trainer = Trainer(
    #fast_dev_run=True,
    logger=wandb_logger,
    # default_root_dir='..',
    val_check_interval=0.1,
    callbacks=[checkpoint_callback, lr_callback],
)

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
model = models.future_model.LitFutureModel(FatNeck)
wandb_logger.watch(model.future_neck, log='all')

wandb: Currently logged in as: wilswu. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [9]:
trainer.fit(model=model, train_dataloaders=loaders['train'], val_dataloaders=loaders['val'])

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/loops/utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:630: Checkpoint directory /checkpoints/ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=5` in the `DataLoader` to improve performance.

  | Name        | Type             | Params
-------------------------------------------------
0 | base_model  | GPT2LMHeadModel  | 124 M 
1 | future_neck | LSTMNeck         | 1.1 M 
2 | loss_func   | CrossEntropyLoss | 0     
----

Sanity Checking: |                                                                                            …

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=5` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:211: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.


Validation: |                                                                                                 …

Validation: |                                                                                                 …

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [10]:
import matplotlib.pyplot as plt
#neck = model.future_neck.weight.data
neck = np.load('/checkpoints/neck.npy')
plt.imshow(np.log(neck))

NameError: name 'np' is not defined

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Error while calling W&B API: failed to find run LAISR_FutureGPT2/bvz047e2 (<Response [404]>)
[rank: 0] Received SIGTERM: 15
wandb: ERROR Error while calling W&B API: run LAISR_FutureGPT2/bvz047e2 not found during createRunFiles (<Response [404]>)
wandb: ERROR Error while calling W&B API: run LAISR_FutureGPT2/bvz047e2 not found during createRunFiles (<Response [404]>)
wandb: ERROR Error while calling W&B API: run LAISR_FutureGPT2/bvz047e2 not found during createRunFiles (<Response [404]>)
wandb: ERROR Error while calling W&B API: run LAISR_FutureGPT2/bvz047e2 not found during createRunFiles (<Response [404]>)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
plt.imshow(neck)

In [ ]:
neck

In [ ]:
import numpy as np
#np.save('/checkpoints/neck.npy', neck.cpu())

In [ ]:
ckpt_model = models.future_model.LitFutureModel.load_from_checkpoint('/home/wwu/FutureGPT2/src/LAISR_FutureGPT2/8wd5g5pv/checkpoints/epoch=4-step=3125.ckpt')

In [ ]:
ckpt_model.future_neck.weight.data